In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from dataset import QuickDrawDataset
from utils import AbsolutePenPositionTokenizer
from tqdm import tqdm
import pickle

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

seed = 42
torch.manual_seed(seed)
if device == "cuda":
    torch.cuda.manual_seed_all(seed)

Using device: cuda


In [3]:
labels = ["cat"]

training_data = QuickDrawDataset(
    labels=labels,
)

tokenizer = AbsolutePenPositionTokenizer(bins=64)

class SketchDataset(Dataset):
    def __init__(self, svg_list, tokenizer, max_len=200, cache_file="sketch_tokenized_dataset.pkl"):
        self.data = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad_id = tokenizer.vocab["PAD"]
        
        # Try to load from cache
        try:
            with open(cache_file, "rb") as f:
                self.data = pickle.load(f)
            print(f"Loaded tokenized data from {cache_file}")
        except FileNotFoundError:
            for svg in tqdm(svg_list, desc="Tokenizing SVGs"):
                tokens = tokenizer.encode(svg)
                # Truncate + Pad
                tokens = tokens[:max_len]
                tokens = tokens + [self.pad_id] * (max_len - len(tokens))
                self.data.append(tokens)
                
            # Save to cache
            with open(cache_file, "wb") as f:
                pickle.dump(self.data, f)
            print(f"Saved tokenized data to {cache_file}")

    def __getitem__(self, idx):
        seq = self.data[idx]
        input_ids = torch.tensor(seq[:-1])
        target_ids = torch.tensor(seq[1:])
        return input_ids, target_ids
    
    def __len__(self):
        return len(self.data)
    
dataset = SketchDataset(training_data, tokenizer, max_len=200)

Loading QuickDraw files: 100%|██████████| 1/1 [00:03<00:00,  3.18s/it]


Loaded tokenized data from sketch_tokenized_dataset.pkl


In [4]:
def generate_square_subsequent_mask(sz: int):
    """Causal mask to stop attention to future positions"""
    return torch.triu(torch.ones(sz, sz), diagonal=1).bool()

class SketchTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=8, num_layers=6, max_len=200):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.max_len = max_len

        self.embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = nn.Embedding(max_len, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=4*d_model
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        """
        x: (batch, seq_len) input tokens
        Returns: (batch, seq_len, vocab_size) logits
        """
        batch_size, seq_len = x.shape
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0)

        x = self.embed(x) + self.pos_embed(positions)  # (batch, seq_len, d_model)
        x = x.transpose(0, 1)  # -> (seq_len, batch, d_model)

        # causal mask (seq_len, seq_len)
        mask = generate_square_subsequent_mask(seq_len).to(x.device)

        x = self.transformer(x, mask=mask)  # (seq_len, batch, d_model)
        x = x.transpose(0, 1)  # back to (batch, seq_len, d_model)

        logits = self.fc_out(x)  # (batch, seq_len, vocab_size)
        return logits
    
def train_model(model, dataloader, vocab_size, epochs=10, lr=1e-4, device="cuda"):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # ignore pad token

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for input_ids, target_ids in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            input_ids, target_ids = input_ids.to(device), target_ids.to(device)

            # Transformer expects shape (seq_len, batch, d_model)
            logits = model(input_ids)  # (seq_len, batch, vocab_size)
            loss = criterion(logits.view(-1, vocab_size), target_ids.view(-1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1} Loss: {total_loss/len(dataloader):.4f}")


dataloader = DataLoader(dataset, batch_size=256, shuffle=True, pin_memory=True)
model = SketchTransformer(vocab_size=len(tokenizer.vocab), d_model=256, nhead=8, num_layers=6)
model = torch.load("sketch_transformer_cat_checkpoint0.pth", weights_only=False)

train_model(model, dataloader, vocab_size=len(tokenizer.vocab), epochs=40, lr=1e-4, device=device)

c:\Users\matth\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
Epoch 1/40: 100%|██████████| 403/403 [01:32<00:00,  4.35it/s]


Epoch 1 Loss: 1.5214


Epoch 2/40: 100%|██████████| 403/403 [01:32<00:00,  4.34it/s]


Epoch 2 Loss: 1.5168


Epoch 3/40: 100%|██████████| 403/403 [01:33<00:00,  4.33it/s]


...

Epoch 5 Loss: 1.5065


Epoch 6/40: 100%|██████████| 403/403 [01:33<00:00,  4.31it/s]


Epoch 6 Loss: 1.5036


Epoch 7/40:   3%|▎         | 14/403 [00:03<01:36,  4.01it/s]


KeyboardInterrupt: 

In [ ]:
# save the model
torch.save(model, "sketch_transformer_cat_checkpoint1.pth")

In [ ]:
# model = torch.load("sketch_transformer.pth", map_location=device, weights_only=False)

def sample_sequence(model, start_token, max_len=200, temperature=1.0, greedy=False, eos_id=None, device="cuda"):
    model.eval()
    
    tokens = [start_token]
    tokens_tensor = torch.tensor([tokens], device=device)  # (1, 1)

    for _ in range(max_len - 1):
        with torch.no_grad():
            logits = model(tokens_tensor)  # (batch=1, seq_len, vocab_size)
            next_logits = logits[:, -1, :] / temperature  # take last step
            probs = F.softmax(next_logits, dim=-1)

            if greedy:
                next_token = torch.argmax(probs, dim=-1).item()
            else:
                next_token = torch.multinomial(probs, num_samples=1).item()

        tokens.append(next_token)

        # stop if EOS reached
        if eos_id is not None and next_token == eos_id:
            break

        next_token_tensor = torch.tensor([[next_token]], device=device)
        tokens_tensor = torch.cat([tokens_tensor, next_token_tensor], dim=1)

    return tokens

# TODO Top-k filtering, Top-p filtering (nucleus)

start_token = tokenizer.vocab["START"]
eos_token = tokenizer.vocab.get("END", None)

generated = sample_sequence(
    model, 
    start_token, 
    max_len=200, 
    temperature=0.5, 
    greedy=False, 
    eos_id=eos_token, 
    device=device
)

print("Generated token sequence:", generated)
decoded_sketch = tokenizer.decode(generated)
print("Decoded sketch:", decoded_sketch)

from IPython.display import HTML, display

display(HTML(f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Generated</b><br>{decoded_sketch}</div>'))

Generated token sequence: [4098, 4096, 907, 1344, 1411, 1548, 4096, 1354, 2250, 4096, 2314, 2437, 2624, 2951, 4096, 780, 527, 404, 537, 861, 1312, 1632, 2079, 2333, 2650, 2902, 3088, 3020, 2887, 4096, 1360, 1297, 4096, 2000, 2065, 4096, 1429, 1558, 1749, 1879, 1942, 4096, 1365, 18, 4096, 1111, 793, 29, 4096, 1306, 1114, 735, 4096, 2582, 4050, 4096, 2584, 3929, 4096, 2393, 3614, 4099]
Decoded sketch: <svg viewBox="0 0 64 64"><g stroke-width="0.8">
<path d="M 14 11 L 21 0 L 22 3 L 24 12" stroke="black" fill="none"/>
<path d="M 21 10 L 35 10" stroke="black" fill="none"/>
<path d="M 36 10 L 38 5 L 41 0 L 46 7" stroke="black" fill="none"/>
<path d="M 12 12 L 8 15 L 6 20 L 8 25 L 13 29 L 20 32 L 25 32 L 32 31 L 36 29 L 41 26 L 45 22 L 48 16 L 47 12 L 45 7" stroke="black" fill="none"/>
<path d="M 21 16 L 20 17" stroke="black" fill="none"/>
<path d="M 31 16 L 32 17" stroke="black" fill="none"/>
<path d="M 22 21 L 24 22 L 27 21 L 29 23 L 30 22" stroke="black" fill="none"/>
<path d="M 21 21 L 0 